In [89]:
## ABRIR DATOS Y AÑADIR LAS COORDENADAS ##

import pickle
import time
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

# Cargar el dataset 
with open("dataset_dhl.pkl", "rb") as f:
    data = pickle.load(f)

# Metemos las coordenadas que ya conocemos (las que no podemos calcular con geopy)
coordenadas_fijas = {
    "Atlante - Atlante - Nieves Sant Esteve": (41.504070650751906, 1.883253906745552),
    "Atlante - Nieves Zona Franca":( 41.33194049366559, 2.134952349708804),
    "IONITY Montcada i Reixac": (41.491422605263075, 2.1823508377517684),
    "IONITY Barcera del Valles": (41.51182358577195, 2.1345502627626303),
    "Ctra. Molins de Rei, 251, Km. 8, 08191 Rubí, Barcelona": (41.46930679260032, 2.029808746545829),
    "Carrer Sakura, 16B, NAVE 16A, 08272 Sant Fruitós de Bages, Barcelona": (41.77803590470732, 1.867141253552206),
    "Avinguda Can Montcau, 1-3-5, 08186 Lliçà d'Amunt, Barcelona": (41.60289038509304, 2.2569771240231096),
    "DHL FREIGHT SPAIN S.L, C/ Atlantic 109-111  (ZAL)": (41.548266507563135, 2.1687455645705684)
}

# Inicializar geolocalizador 
geolocator = Nominatim(user_agent="geoapi")

# Añadir coordenadas 
for d in data:
    direccion = d.get("direccion", "").strip()
    nombre = d.get("nombre", "").strip()

    # Caso 1: coordenadas conocidas
    if direccion in coordenadas_fijas:
        d["coordenadas"] = coordenadas_fijas[direccion]

    elif nombre in coordenadas_fijas:
        d["coordenadas"] = coordenadas_fijas[nombre]

    # Caso 2: intentar buscar con geopy
    elif direccion:
        try:
            location = geolocator.geocode(direccion)
            if location:
                d["coordenadas"] = (location.latitude, location.longitude)
            else:
                d["coordenadas"] = None
                print(f"No se encontró la dirección: {nombre}")
        except Exception as e:
            print(f"Error al buscar {nombre}: {e}")
            d["coordenadas"] = None
        time.sleep(1)  # Evita bloqueo por exceso de peticiones

    else:
        d["coordenadas"] = None
        print(f" No hay dirección para: {nombre}")

#Calcular matriz de distancias 
n = len(data)
distancias = [[0]*n for _ in range(n)]

for i in range(n):
    for j in range(n):
        if i != j and data[i]["coordenadas"] and data[j]["coordenadas"]:
            dist = geodesic(data[i]["coordenadas"], data[j]["coordenadas"]).km
            distancias[i][j] = round(dist, 2)
        else:
            distancias[i][j] = None if i != j else 0


#Mostrar matriz
for fila in distancias:
    print(fila)





[0, 16.5, 36.87, 24.07, 24.27, 14.54, 5.92, 9.94, 12.98, 17.61, 12.83]
[16.5, 0, 52.68, 36.42, 37.57, 27.64, 11.39, 22.8, 22.62, 8.48, 23.98]
[36.87, 52.68, 0, 37.84, 35.24, 35.81, 41.3, 37.02, 41.27, 54.35, 30.46]
[24.07, 36.42, 37.84, 0, 2.81, 9.54, 29.41, 14.37, 13.86, 31.77, 33.05]
[24.27, 37.57, 35.24, 2.81, 0, 10.05, 29.85, 15.0, 15.3, 33.43, 32.3]
[14.54, 27.64, 35.81, 9.54, 10.05, 0, 19.9, 4.95, 6.41, 24.19, 24.33]
[5.92, 11.39, 41.3, 29.41, 29.85, 19.9, 0, 15.04, 17.02, 14.83, 13.6]
[9.94, 22.8, 37.02, 14.37, 15.0, 4.95, 15.04, 0, 4.59, 19.98, 21.0]
[12.98, 22.62, 41.27, 13.86, 15.3, 6.41, 17.02, 4.59, 0, 18.15, 25.01]
[17.61, 8.48, 54.35, 31.77, 33.43, 24.19, 14.83, 19.98, 18.15, 0, 28.43]
[12.83, 23.98, 30.46, 33.05, 32.3, 24.33, 13.6, 21.0, 25.01, 28.43, 0]


In [125]:
## CREAR EL CONJUNTO DE NODOS ##
V = list(range(len(data)))
N = [i for i,d in enumerate(data) if d['nodo']=="carga"]
F = [i for i,d in enumerate(data) if d['nodo']=="cargador"]
base_operativa = [i for i,d in enumerate(data) if d['nodo']=="parking"][0]
descarga = [i for i,d in enumerate(data) if d['nodo']=="descarga"][0]

print(V)
print(N)
print(F)
print(base_operativa)
print(descarga)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 2, 3, 4]
[6, 7, 8, 9, 10]
0
5


In [91]:
# Nodos de mi Data Set
[{'nodo': 'parking', 'nombre': 'ES08191 Rubi', 'direccion': 'Ctra. Molins de Rei, 251, Km. 8, 08191 Rubí, Barcelona'},
{'nodo': 'carga', 'nombre': 'ES08840 Viladecans', 'direccion': 'Av. del Segle XXI, 63, 08840 Viladecans, Barcelona'},
{'nodo': 'carga', 'nombre': 'ES08272 Sant Fruitós de Bages', 'direccion': 'Carrer Sakura, 16B, NAVE 16A, 08272 Sant Fruitós de Bages, Barcelona'}, 
{'nodo': 'carga', 'nombre': 'ES08650 Sallent', 'direccion': "Avinguda Can Montcau, 1-3-5, 08186 Lliçà d'Amunt, Barcelona"}, 
{'nodo': 'carga', 'nombre': 'ES08186 Lliça de Amunt', 'direccion': "Carrer d'Anselm Clavé, 62 AB, 08186 Lliçà d'Amunt, Barcelona"}, 
{'nodo': 'descarga', 'nombre': 'ES08040 Barcelona', 'direccion': 'DHL FREIGHT SPAIN S.L, C/ Atlantic 109-111  (ZAL)'},
{'nodo': 'cargador', 'nombre': 'IONITY Palleja', 'direccion': 'Ronda de Santa Eulàlia, 31, 08780 Pallejà, Barcelona'}, 
{'nodo': 'cargador', 'nombre': 'IONITY Barcera del Valles', 'direccion': ''},
{'nodo': 'cargador', 'nombre': 'IONITY Montcada i Reixac', 'direccion': ''},
{'nodo': 'cargador', 'nombre': 'Atlante - Nieves Zona Franca', 'direccion': ''}, 
{'nodo': 'cargador', 'nombre': 'Atlante - Atlante - Nieves Sant Esteve', 'direccion': ''}]



[{'nodo': 'parking',
  'nombre': 'ES08191 Rubi',
  'direccion': 'Ctra. Molins de Rei, 251, Km. 8, 08191 Rubí, Barcelona'},
 {'nodo': 'carga',
  'nombre': 'ES08840 Viladecans',
  'direccion': 'Av. del Segle XXI, 63, 08840 Viladecans, Barcelona'},
 {'nodo': 'carga',
  'nombre': 'ES08272 Sant Fruitós de Bages',
  'direccion': 'Carrer Sakura, 16B, NAVE 16A, 08272 Sant Fruitós de Bages, Barcelona'},
 {'nodo': 'carga',
  'nombre': 'ES08650 Sallent',
  'direccion': "Avinguda Can Montcau, 1-3-5, 08186 Lliçà d'Amunt, Barcelona"},
 {'nodo': 'carga',
  'nombre': 'ES08186 Lliça de Amunt',
  'direccion': "Carrer d'Anselm Clavé, 62 AB, 08186 Lliçà d'Amunt, Barcelona"},
 {'nodo': 'descarga',
  'nombre': 'ES08040 Barcelona',
  'direccion': 'DHL FREIGHT SPAIN S.L, C/ Atlantic 109-111  (ZAL)'},
 {'nodo': 'cargador',
  'nombre': 'IONITY Palleja',
  'direccion': 'Ronda de Santa Eulàlia, 31, 08780 Pallejà, Barcelona'},
 {'nodo': 'cargador', 'nombre': 'IONITY Barcera del Valles', 'direccion': ''},
 {'nodo':

In [ ]:
## CALCULO DE VARIABLES ##
# Calculo de d (tiempos):
import openrouteservice

client = openrouteservice.Client(key='eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6IjEyZmEwNWY4YmYwNjRmNjZiY2UwMGE4M2RiNGJiNmUzIiwiaCI6Im11cm11cjY0In0=')
n = len(data)
tiempos = [[0]*n for _ in range(n)]

for i in range(n):
    for j in range(n):
        if i != j and data[i]["coordenadas"] and data[j]["coordenadas"]:
            coords = [data[i]["coordenadas"][::-1], data[j]["coordenadas"][::-1]]  # (lon, lat)
            try:
                ruta = client.directions(coords, profile='driving-car')
                duracion = ruta['routes'][0]['summary']['duration'] / 60  # segundos → minutos
                tiempos[i][j] = round(duracion, 2)
            except Exception as e:
                print(f"Error al calcular tiempo entre {data[i]['nombre']} y {data[j]['nombre']}: {e}")
                tiempos[i][j] = None
            time.sleep(1)  # evita saturar el servidor
        else:
            tiempos[i][j] = 0 if i == j else None

# Mostrar la matriz
for fila in tiempos:
    print(fila)



KeyboardInterrupt: 

In [ ]:
# Vamos a suponer que el camion va a una velocidad media de 70km/h
tiempos = [[0]*n for _ in range(n)]
velocidad=70
for i in range(len(distancias)):
    for j in range(len(distancias)):
        tiempos[i][j]=distancias[i][j]/70*60 # en minutos
for filas in tiempos:
    print(filas)   






[0.0, 14.142857142857142, 31.60285714285714, 20.63142857142857, 20.802857142857142, 12.462857142857143, 5.074285714285715, 8.52, 11.125714285714286, 15.094285714285714, 10.997142857142856]
[14.142857142857142, 0.0, 45.15428571428571, 31.21714285714286, 32.20285714285714, 23.69142857142857, 9.762857142857143, 19.542857142857144, 19.38857142857143, 7.268571428571429, 20.554285714285715]
[31.60285714285714, 45.15428571428571, 0.0, 32.434285714285714, 30.205714285714286, 30.694285714285712, 35.4, 31.731428571428573, 35.37428571428572, 46.58571428571429, 26.10857142857143]
[20.63142857142857, 31.21714285714286, 32.434285714285714, 0.0, 2.408571428571429, 8.177142857142856, 25.208571428571428, 12.317142857142857, 11.879999999999999, 27.23142857142857, 28.328571428571426]
[20.802857142857142, 32.20285714285714, 30.205714285714286, 2.408571428571429, 0.0, 8.614285714285714, 25.585714285714285, 12.857142857142856, 13.114285714285716, 28.654285714285717, 27.685714285714283]
[12.462857142857143, 

In [123]:
# Precios de los cargadores
import re
precio_ionity = 0.61
precio_atlante = 0.64
kwh = 300
#for d in data:
#    if re.search("IONITY",d["nombre"]):
#        d["compañia del cargador"]="IONITY"
#    elif re.search("Atlante",d["nombre"]):
#        d["compañia del cargador"]="Atlante"

w=[]
for d in data:
    if re.search("IONITY",d["nombre"]):
        w.append(kwh*precio_ionity)
    elif re.search("Atlante",d["nombre"]):
        w.append(kwh*precio_atlante)
    else:
        w.append(0)
print(w)
type(w)




[0, 0, 0, 0, 0, 0, 183.0, 183.0, 183.0, 192.0, 192.0]


list

In [111]:
# Calculamos la matriz cij, coste de ir de i a j
# cij=dist*precio
precio=0.4
c=[[0]*len(data) for _ in range(len(data))]
for i in range(len(data)):
    for j in range(len(data)):
        c[i][j]=distancias[i][j]*0.4
print(c)

        


[[0.0, 6.6000000000000005, 14.748, 9.628, 9.708, 5.816, 2.368, 3.976, 5.192, 7.0440000000000005, 5.132000000000001], [6.6000000000000005, 0.0, 21.072000000000003, 14.568000000000001, 15.028, 11.056000000000001, 4.556, 9.120000000000001, 9.048, 3.3920000000000003, 9.592], [14.748, 21.072000000000003, 0.0, 15.136000000000003, 14.096000000000002, 14.324000000000002, 16.52, 14.808000000000002, 16.508000000000003, 21.740000000000002, 12.184000000000001], [9.628, 14.568000000000001, 15.136000000000003, 0.0, 1.124, 3.816, 11.764000000000001, 5.748, 5.5440000000000005, 12.708, 13.219999999999999], [9.708, 15.028, 14.096000000000002, 1.124, 0.0, 4.0200000000000005, 11.940000000000001, 6.0, 6.120000000000001, 13.372, 12.92], [5.816, 11.056000000000001, 14.324000000000002, 3.816, 4.0200000000000005, 0.0, 7.96, 1.9800000000000002, 2.564, 9.676000000000002, 9.732], [2.368, 4.556, 16.52, 11.764000000000001, 11.940000000000001, 7.96, 0.0, 6.016, 6.808, 5.932, 5.44], [3.976, 9.120000000000001, 14.8080

In [145]:
# Definimos la M grande
M = 1000

In [148]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# Pasamos las variables que son listas a arrays
c = np.array(c)
w = np.array(w)
tiempos = np.array(tiempos)



# Creamos el modelo
m = gp.Model("problema1")

# Creamos las variables
x = m.addVars(V,V, vtype=GRB.BINARY, name="x")
z = m.addVars(F, vtype=GRB.BINARY, name="z")
t = m.addVars(V, lb=0.0, ub= 15*60, vtype=GRB.CONTINUOUS, name="t")
m.update()

# Definimos la función objetivo
obj = gp.quicksum(c[i, j] * x[i, j] for i in V for j in V if i != j) + gp.quicksum(w[i] * z[i] for i in F)
m.setObjective(obj, GRB.MINIMIZE)

# Definimos las restricciones
## Ponemos que en la base operattiva el tiempo es =0
#m.addConstr(t[base_operativa] == 0, name="r0")
# r1
for i in V:
    if i not in F:
        m.addConstr(gp.quicksum(x[i, j] for j in V if j != i) == 1, name="r1")
#r2
m.addConstr(gp.quicksum(x[base_operativa,j] for j in V if j!=0)==1, name="r2")
#r3
m.addConstr(x[descarga, base_operativa]==1, name ="r3")
#r4
for j in V:
    m.addConstr(gp.quicksum(x[i,j] for i in V)-gp.quicksum(x[j,h] for h in V)==0, name="r4")
#r5
for i in V:
    for j in V:
        m.addConstr(t[i]+tiempos[i,j]*x[i,j]-t[j]<=(1-x[i,j])*M, name="r5")     
#r6
m.addConstr(gp.quicksum(x[i,j] for i in (N+[base_operativa]) for j in F)==1, name="r6")
#r7
for i in F:
    for j in N:
        m.addConstr(x[i,j]<= z[i], name="r7")        
#r8
for i in N:
    for j in F:
        m.addConstr(x[i,j]<= z[j], name="r8") 

m.update()
m.optimize()
a= m.ObjVal
print("Funcion objetivo", a)
for v in m.getVars():
    print(str(v.VarName)+"="+str(round(v.x, 2)))


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1123)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 181 rows, 137 columns and 737 nonzeros
Model fingerprint: 0x1c9b8507
Variable types: 11 continuous, 126 integer (126 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 2e+02]
  Bounds range     [1e+00, 9e+02]
  RHS range        [1e+00, 1e+03]
Presolve removed 50 rows and 32 columns
Presolve time: 0.01s
Presolved: 131 rows, 105 columns, 640 nonzeros
Variable types: 10 continuous, 95 integer (95 binary)
Found heuristic solution: objective 1014.7120000
Found heuristic solution: objective 253.9520000

Root relaxation: objective 5.618067e+01, 34 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd 

In [ ]:
print(F)
print(w)
range(len(F))

[6, 7, 8, 9, 10]
[183. 183. 183. 192. 192.]


range(0, 5)